In [2]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [3]:
HF_TOKEN = os.getenv("HF_TOKEN")
HF_TOKEN.startswith("hf_")

True

## Dataset Preprocessing

In [4]:
raw_datasets = load_dataset("glue", "mrpc")  # Loads the glue dataset's MRPC subset, stores it in ~/.cache/huggingface/datasets/glue/mrpc
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Using the latest cached version of the dataset since glue couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'mrpc' at /Users/thomas/.cache/huggingface/datasets/glue/mrpc/0.0.0/bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c (last modified on Sun Mar  2 20:51:21 2025).


In [5]:
tokenized_datasets['train']

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3668
})

In [6]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"]) # not expected by the model
tokenized_datasets = tokenized_datasets.rename_column("label", "labels") # again to meet the model's expectations
tokenized_datasets.set_format("torch") # we want PyTorch tensors

In [7]:
tokenized_datasets["train"]

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3668
})

## Dataloader

In [8]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [9]:
batch = next(iter(train_dataloader))
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 89]),
 'token_type_ids': torch.Size([8, 89]),
 'attention_mask': torch.Size([8, 89])}

## Training

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.7166, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [12]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [13]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


In [14]:
import torch

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='mps')

In [15]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        # batch = {k: v.to(device) for k, v in batch.items()}
        batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [18]:
# import evaluate

# metric = evaluate.load("glue", "mrpc")
# model.eval()
# for batch in eval_dataloader:
#     # batch = {k: v.to(device) for k, v in batch.items()}
#     batch.to(device)
#     with torch.no_grad():
#         outputs = model(**batch)

#     logits = outputs.logits
#     predictions = torch.argmax(logits, dim=-1)
#     metric.add_batch(predictions=predictions, references=batch["labels"])

# metric.compute()

# getting "FileNotFoundError: Couldn't find a module script at ~/code/hf-agents-course/trian_fine_tune_llms/glue/glue.py. Module 'glue' doesn't exist on the Hugging Face Hub either."

In [22]:
from sklearn.metrics import accuracy_score, f1_score

model.eval()
all_predictions = []
all_labels = []

for batch in eval_dataloader:
    batch.to(device)
    with torch.no_grad():
        outputs = model(**batch)

    predictions = torch.argmax(outputs.logits, dim=-1)
    all_predictions.extend(predictions.cpu().numpy())
    all_labels.extend(batch["labels"].cpu().numpy())

accuracy = accuracy_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8505
F1 Score: 0.8939
